# Do wildfires burn up jobs in rural California? 
# Investigating the impact of wildifre size on rural economies

## Overview
### Motivation
Global climate change is driving increases in wildfire occurence and intensity, while increasing anthropogenic land use brings humans in more contact with wilderness areas with greater fire risks. Major Californian wildfires have garnered global news attention and caused great financial dammage and human suffering. Understanding the effects of wildfire on humans and economies on local/regional scales will aid in better planning for a future where risk of wildfire is higher. Here, I invesitgate the relationship between wildfire area and socioeconomic in rural counties in California.

### Hypotheses (Incomplete)
Wildfires are devastating natural disasters that not only dirupt the lives and economies of the communities where they occur, but may also have significant post-fire effects that prevent a return to normalacy (which itself has cascading effects). Rural californian counties that experience higher wildfire occurrence may experience decreases in population and  employment rates following the event. I test the following hypotheses:
1) 
2) 
3)

### Data Sources
Socioeconomic data at the county level is available from the [Atlas of Rural and Small Town America](https://catalog.data.gov/dataset/atlas-of-rural-and-small-town-america) (Economic Research Service, United States Department of Agriculture, 2021). Wildfire data was drawn from the Kaggle dataset [California WildFires (2013-2020)](https://www.kaggle.com/ananthu017/california-wildfire-incidents-20132020?select=California_Fire_Incidents.csv) (Kaggle User Ares, 2020).

### Methods (Incomplete)
First, I prepared the socioeconomic statistics and population estimate tables.


## Data Preparation

I begin by improtign the Python libraries I will use in this analysis, along with the datasets I will use.

In [148]:
# Import libraries
import pandas as pd
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

### Clean and format Rural Atlas of America data

First, I import all three datasets I will be using (see Overview: Data Sources for more information) and clean the Rural Atlas of America datasets to include only Californian data.

In [149]:
# Import Rural Atlas population estimates
people = pd.read_csv('people.csv')  

# Import Rural Atlas employment metrics.
jobs = pd.read_csv('jobs.csv') 

# Import California Wildfire dataset.
fires = pd.read_csv('California_Fire_Incidents.csv') 

Note that five variables in the jobs.csv dataset are inconsitently named and have an extra letter capitalized (e.g. NumCivLaborforce2011 vs NumCivLaborForce2011). The affected variables are: 'NumCivLaborforce2011', 'NumCivLaborforce2009', 'NumCivLaborforce2012', 'NumCivLaborforce2008', and 'NumCivLaborforce2010'. These were fixed manually by editing the jobs.csv data table in MS Excel.

In [150]:
# Select only counties in California for the:
# jobs dataset,
jobs = jobs[jobs['State'] == 'CA'] 
# and the people dataset.
people = people[people['State'] == 'CA']

By creating a helper function, separateData(), I can retrieve a useful subset of data in each Rural Atlas of America dataset and place it in a new dataframe for further manipulation and analysis. 

In [151]:
def separateData(df, lst, y1, y2):
    """Select data from main table using a list of row names and transpose.

    df == parent dataframe (array)
    lst == a list of column names to select from the df (list)
    y1 == the first year of data available
    y2 == the last year of data available
    """
    
    dfname = df[lst]
    # Remove State column.
    dfname = dfname.drop('State', 1) 
    # Make County column new index.
    dfname = dfname.set_index('County')
    # Transpose so columns are names while rows are years. 
    dfname = dfname.transpose()

    # Initialize a list of years for the specified time period.
    years = list(range(y1, y2 + 1)) 
    # Format values as strings.
    years = [str(x) for x in years]
    # Zip years with the matching column name into a dictionary.
    indices = dict(zip(lst[2:], years))

    # Rename the index column with the year values in the indices dictionary.
    dfname.rename(index=indices, inplace=True)

    # Return the dataframe.
    return dfname

I use another helper function to prepare strings of columns names to be used as input for the lst variable for the separateData() function.

In [152]:
def colNameList(name, y1, y2):
    """Generate a list of variable names for repeating years.

    name == the base variable name.
    y1 == the first year of data available.
    y2 == the last year of data available.
    """
    
    # Initialize a list with preset first two values.
    lst = ['State', 'County']
    # Initialize a list of the years this function produces variables for.
    years = list(range(y1, y2 + 1))
    
    # Repeat for each year.
    for x in years:
        # Combine name and year into one string.
        value = str(name) + str(x)
        # Append combined name-year string to the list.
        lst.append(value)
    
    # Return the list
    return lst

In [153]:
# Use the helper function colNameList() to initialize lists of names for:
# Population estimates,
population = colNameList('TotalPopEst', 2010, 2019)
# Civilian labor force size,
labor = colNameList('NumCivLaborforce', 2007, 2020)
# Number of employed people,
emp = colNameList('NumEmployed', 2007, 2020)
# Unemplyment rates, and
unempr = colNameList('UnempRate', 2007, 2020)
# Number of unemployed people.
unemp = colNameList('NumUnemployed', 2007, 2020)

In [154]:
# Use the helper function separateData() to separate and format tables for:
# Population estimates, 
npop = separateData(people, population, 2010, 2019)
# Civilian labor force size,
nlabor = separateData(jobs, labor, 2007, 2020)
# Number of employed people,
nemp = separateData(jobs, emp, 2007, 2020)
# Unemplyment rates, and
runemp = separateData(jobs, unempr, 2007, 2020)
# Number of unemployed people.
numemp = separateData(jobs, unemp, 2007, 2020)

#### Clean and prepare wildfire dataset

In [155]:
# Drop rows for States of Oregon, Nevada, and Mexico
fires = fires[fires['Counties'] != 'State of Oregon'] 
fires = fires[fires['Counties'] != 'State of Nevada']
fires = fires[fires['Counties'] != 'Mexico']
fires = fires[fires['ArchiveYear'] < 2019] # so 2018 fires match next years' data (2019)

# Calculate acres burned per county per year and format
acres_burned = fires.groupby(["Counties", "ArchiveYear"])["AcresBurned"].sum().to_frame(name = 'AcresBurned')

# Reset indexes so dataframe is singly indexed
acres_burned = acres_burned.reset_index()

### Associate year-after employment statistics with yearly wildfire data

Here, I associate employment statistics with wildfire data by creating a helper function to gather year-after-fire data and year/ year-after-fire difference data for each year and county combination (e.g., Yuba county, 2017) and attach these as a new column in the acres_burned dataset.

In [156]:
# Create a helper function to gather year-after-fire data and year/ year-after-fire differences.

def yearAfterStats(df1, df2, colname):
    """Get year-after-fire data and calculate year/ year-after-fire differences from df2 for each year-county combo in df1 and attach as new column named 'colname' to df1."""
    
    # Catch year after fire values in a list
    yafLst = []
    # Catch year-of/year-after fire difference
    yafDiffLst = []
    # Catch year-before/year-of fire difference
    yofDiffLst = []

    # Iterate over each row in df1
    for index, row in df1.iterrows():
        # Get the county
        county = str(row[0])
        
        # Get the year before fire
        ybf = str(row[1] - 1)
        # Get the year of fire
        yof = str(row[1])
        # Get the year after fire
        yaf = str(row[1] + 1)
        
        # Get labor stat from the year before fire/county combo from df2
        ybfVal = df2.loc[ybf, county]
        # Get labor stat from the year of fire/county combo from df2
        yofVal = df2.loc[yof, county]
        # Get labor stat from the year after fire/county combo from df2 
        yafVal = df2.loc[yaf, county]
        
        # Calculate difference year-of and year-after values
        yafDiff = yofVal - yafVal
        # Calculate difference year-before and year-of values
        yofDiff = ybfVal - yofVal

        # Append year-after values to the appropriate list.
        yafLst.append(yafVal)
        # Append year-after values to the appropriate list.
        yafDiffLst.append(yafDiff)
        # Append year-after values to the appropriate list.
        yofDiffLst.append(yofDiff)

    # Create full column name
    colname_yaf = colname + 'Yaf'
    # Attach to dataframe
    df1[colname_yaf] = yafLst

    # Create full column name
    colname_yafdiff = colname + 'ChangeYaf'
    # Attach to dataframe
    df1[colname_yafdiff] = yafDiffLst

    # Create full column name
    colname_yofdiff = colname + 'ChangeYof'
    # Attach to dataframe
    df1[colname_yofdiff] = yofDiffLst  
    
      

In [157]:
yearAfterStats(acres_burned, nlabor, 'nLabor')
yearAfterStats(acres_burned, nemp, 'nEmp')
yearAfterStats(acres_burned, nunemp, 'nUnemp')
yearAfterStats(acres_burned, runemp, 'rUnemp')
yearAfterStats(acres_burned, population, 'nPop')

NameError: name 'nunemp' is not defined

In [ ]:
acres_burned


## Data Exploration

### Normality

Many statistical methods (e.g., parametric statistical tests and many machine learning methods) assume variables are normally distributed; if this assumption is violated, these methods may produce inaccurate and misleading results. Non-normal data may be transformed or analyzed with non-parametric tests.

To determine if the variables examined here follow a normal distribution, I begin with a visual examination of the data and follow by applying Shapiro-Wilkes tests of normality.

In [ ]:
# Select only continuous data
burn_data = acres_burned.drop(['Counties', 'ArchiveYear'], axis=1)
# Plot variables against each other
sns.pairplot(burn_data) # Diagonal (top-left to bottom-right) are histograms of 


Visual inspection of the diagonal histograms (above) indicates none of the numeric data are normally distributed.

In [ ]:
variable = []
pval = []
shapiro_W = []
for name, values in burn_data.iteritems():
    variable.append(name)
    res = stats.shapiro(values)
    shapiro_W.append(res[0])
    pval.append(res[1])

norm_res = {'variable': variable, 'p-value': pval, 'W-score': shapiro_W}
shapiro_normality = pd.DataFrame(data = norm_res)
shapiro_normality 

Additionally, Shapiro-Wilkes tests for normality (above) indicate none of the data are normally distributed. Here, significant p-values for each variable reject the null hypotheses that the distributions are not statistically different from a normal distribution. 

Apply a log transform to the data

In [ ]:
burn_data1 = acres_burned[['AcresBurned', 'nlabor_yaf', 'nemp_yaf', 'runemp_yaf', 'nunemp_yaf', 'pop_yf', 'pop_yaf', 'deltapop_yaf']]

for name, values in burn_data.iteritems():
    namea = name + '_log'
    vals = []
    for i in values:
        if i > 0:
            vals.append(math.log(i))
        else:
            vals.append(np.NaN)
    burn_data1[namea] = vals
    
burn_data = burn_data1[['AcresBurned', 'nlabor_yaf', 'nemp_yaf', 'runemp_yaf', 'nunemp_yaf', 'pop_yf', 'pop_yaf', 'deltapop_yaf']]
    
burn_data_log = burn_data1[['AcresBurned_log', 'nlabor_yaf_log', 'nemp_yaf_log', 'runemp_yaf_log', 'nunemp_yaf_log', 'pop_yf_log', 'pop_yaf_log', 'deltapop_yaf_log']]
#burn_data_log

Shapiro-Wilkes test for normality of log transformed data are normally distributed- except for runemp_yaf_log.

In [ ]:
variable = []
pval = []
shapiro_W = []
for name, values in burn_data_log.iteritems():
    variable.append(name)
    vals = values[np.logical_not(np.isnan(values))]    
    res = stats.shapiro(vals)
    shapiro_W.append(res[0])
    pval.append(res[1])

norm_res = {'variable': variable, 'p-value': pval, 'W-score': shapiro_W}
shapiro_normality_log = pd.DataFrame(data = norm_res)
shapiro_normality_log

In [ ]:
sns.pairplot(burn_data_log, kind = 'reg')

In [ ]:
plt.scatter(acres_burned['AcresBurned'], acres_burned['nlabor_yaf'], label='Civilian Labor Force (n)')
plt.scatter(acres_burned['AcresBurned'], acres_burned['nemp_yaf'], label='Employed (n)')
plt.scatter(acres_burned['AcresBurned'], acres_burned['nunemp_yaf'], label='Unemployed (n)')

In [ ]:
plt.scatter(acres_burned['AcresBurned'], acres_burned['pop_yaf'], label='Population')

In [ ]:
plt.scatter(acres_burned['AcresBurned'], acres_burned['runemp_yaf'], label='Unemployment Rate')

In [ ]:
plt.scatter(acres_burned['AcresBurned'], acres_burned['deltapop_yaf'], label='Change in Population')